In [2]:
import cv2
import pyrealsense2 as rs
import numpy as np

# Setup

In [3]:
def read_bag_file(file_name):
    pipe = rs.pipeline()
    cfg = rs.config()
    cfg.enable_device_from_file(file_name, repeat_playback=False)
    profile = pipe.start(cfg)
    playback = profile.get_device().as_playback()
    playback.set_real_time(False) # False: no frame drop

    return pipe, cfg, profile, playback

In [4]:
# Read the bag file
file_name = "..\Dataset\_realsense\Data_realsense1/20240322_123504.bag"
pipe, cfg, profile, playback = read_bag_file(file_name)
duration = playback.get_duration()
print("Video duration: ", duration)

Video duration:  0:00:41.604937


In [5]:
# Function to align the depth frame to the color frame
def align_frames(frames):
    align = rs.align(rs.stream.color)
    aligned_frames = align.process(frames)
    aligned_depth = aligned_frames.get_depth_frame()
    aligned_color = aligned_frames.get_color_frame()
    return aligned_depth, aligned_color

In [6]:
colorizer = rs.colorizer()

# Read the full stream
pipe, cfg, profile, playback = read_bag_file(file_name)
num_frames = 0
wait_key = 1

try:
    while True:
        # Wait for a coherent pair of frames: depth and color
        frames = pipe.wait_for_frames()

        # Align the depth frame to color frame so they have the same shape
        aligned_depth, aligned_color = align_frames(frames)
        depth_color_frame = colorizer.colorize(aligned_depth)

        depth_color_image = np.asanyarray(depth_color_frame.get_data())
        color_image = np.asanyarray(aligned_color.get_data())

        # Stack both images horizontally
        images = np.hstack((color_image, depth_color_image))
        cv2.imshow("Images", images)

        key = cv2.waitKey(wait_key)

        # Press esc close the image window
        if key == 27:
            break

        # Press s to write the color_image on disk
        if key == ord('s'):
            cv2.imwrite("./color_images/color_image" + str(num_frames) + ".png", color_image)

        # Press d to view the video frame by frame
        if key == ord('d'):
            if wait_key == 0:
                wait_key = 1
            else:
                wait_key = 0

        num_frames += 1

# Catch exception if the stream is ended
except RuntimeError:
    print("Stream ended")
        
finally:
    # Stop streaming
    cv2.destroyAllWindows()
    pipe.stop()

print("Total number of frames: ", num_frames)

Total number of frames:  123


In [ ]:
# Create a mask to remove the walls on the depth map
#mask = np.zeros(frame_shape, dtype="uint8")
#cv2.rectangle(mask, (0, 316), (223, 480), (255, 255, 255), -1) # left wall
#cv2.rectangle(mask, (0, 316), (223, 480), (255, 255, 255), -1) # right wall
#mask = 255 - mask

# Extract height of the participants

In [7]:
# Setup the aruco detector
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_ARUCO_MIP_36h12)
aruco_params = cv2.aruco.DetectorParameters()

aruco_params.adaptiveThreshConstant = 3 # lower -> more aggresive, default 7
aruco_params.errorCorrectionRate = 0.2 # default 0.6
aruco_params.polygonalApproxAccuracyRate = 0.05 # default 0.03

detector = cv2.aruco.ArucoDetector(aruco_dict, aruco_params) 

In [8]:
camera_height = 2.65 # meters

pipe, cfg, profile, playback = read_bag_file(file_name)
num_frames = 0
wait_key = 1

try:
    while True:
        frames = pipe.wait_for_frames()
        # Align the depth frame to color frame so they have the same shape
        aligned_depth, aligned_color = align_frames(frames)

        color_image = np.asanyarray(aligned_color.get_data())

        output = color_image.copy()
        corners, ids, rejected = detector.detectMarkers(color_image)
        cv2.aruco.drawDetectedMarkers(output, corners, ids)

        if len(corners) > 0:
            # Get the distance value at the center of the marker
            c = corners[0][0]
            center_x = int((c[0][0] + c[1][0] + c[2][0] + c[3][0]) / 4)
            center_y = int((c[0][1] + c[1][1] + c[2][1] + c[3][1]) / 4)
            distance = aligned_depth.get_distance(center_x, center_y)

            participant_height = camera_height - distance # meters

            # Draw the distance value on the image
            cv2.putText(output, "Height: {:.2f} m".format(participant_height), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

            cv2.imwrite("./distance_images/distance_image" + str(num_frames) + ".png", output)

        cv2.imshow("Output", output)
        key = cv2.waitKey(wait_key)

        # Press esc close the image window
        if key == 27:
            break

        # Press d to view the video frame by frame
        if key == ord('d'):
            if wait_key == 0:
                wait_key = 1
            else:
                wait_key = 0

        num_frames += 1
           
# Catch exception if the stream is ended
except RuntimeError:
    print("Stream ended")
        
finally:
    # Stop streaming
    cv2.destroyAllWindows()
    pipe.stop()

Stream ended
